In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd

In [2]:
dataTrain = pd.read_csv("../data/mergeTrain.csv")
dataTest = pd.read_csv("../data/mergeTest.csv")
dataTrain

,Unnamed: 0,round,gf,ga,xg,xga,poss,formation,sh,sot,...,pk,pkatt,venue_code,opp_code,hour,day_code,home_code,target_home,target_away,target_draw
0,0,33,0.0,0.0,1.4,0.8,49.0,4,11.0,2.0,...,0.0,0.0,0,5,13,6,0,0,0,1
1,1,35,2.0,0.0,2.4,0.4,64.0,4,25.0,8.0,...,0.0,0.0,1,4,12,6,0,1,0,0
2,2,37,5.0,2.0,3.5,1.0,67.0,4,30.0,14.0,...,0.0,0.0,0,7,13,6,0,1,0,0
3,3,2,1.0,3.0,0.3,1.8,43.0,4,6.0,3.0,...,0.0,0.0,0,12,14,6,0,0,1,0
4,4,3,2.0,0.0,2.1,0.7,43.0,4,14.0,5.0,...,0.0,0.0,1,17,12,5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,1801,24,1.0,0.0,0.5,0.9,50.0,12,7.0,3.0,...,0.0,0.0,0,8,20,4,19,1,0,0
1802,1802,28,2.0,1.0,1.1,0.3,61.0,12,16.0,6.0,...,0.0,0.0,1,4,12,6,19,1,0,0
1803,1803,29,0.0,2.0,0.9,3.2,38.0,12,15.0,3.0,...,0.0,0.0,0,17,14,6,19,0,1,0
1804,1804,30,0.0,1.0,0.3,0.7,59.0,3,9.0,3.0,...,0.0,0.0,0,7,18,2,19,0,1,0


In [3]:
del dataTrain["Unnamed: 0"]
del dataTest["Unnamed: 0"]

In [5]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   round        1806 non-null   int64  
 1   gf           1806 non-null   float64
 2   ga           1806 non-null   float64
 3   xg           1806 non-null   float64
 4   xga          1806 non-null   float64
 5   poss         1806 non-null   float64
 6   formation    1806 non-null   int64  
 7   sh           1806 non-null   float64
 8   sot          1806 non-null   float64
 9   dist         1806 non-null   float64
 10  fk           1806 non-null   float64
 11  pk           1806 non-null   float64
 12  pkatt        1806 non-null   float64
 13  venue_code   1806 non-null   int64  
 14  opp_code     1806 non-null   int64  
 15  hour         1806 non-null   int64  
 16  day_code     1806 non-null   int64  
 17  home_code    1806 non-null   int64  
 18  target_home  1806 non-null   int64  
 19  target

In [6]:
train_data = dataTrain.iloc[:, :-3]
train_labels = dataTrain.iloc[:, -3:]
train_labels

,target_home,target_away,target_draw
0,0,0,1
1,1,0,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
1801,1,0,0
1802,1,0,0
1803,0,1,0
1804,0,1,0


In [7]:
train_data

,round,gf,ga,xg,xga,poss,formation,sh,sot,dist,fk,pk,pkatt,venue_code,opp_code,hour,day_code,home_code
0,33,0.0,0.0,1.4,0.8,49.0,4,11.0,2.0,17.5,0.0,0.0,0.0,0,5,13,6,0
1,35,2.0,0.0,2.4,0.4,64.0,4,25.0,8.0,18.4,2.0,0.0,0.0,1,4,12,6,0
2,37,5.0,2.0,3.5,1.0,67.0,4,30.0,14.0,16.6,2.0,0.0,0.0,0,7,13,6,0
3,2,1.0,3.0,0.3,1.8,43.0,4,6.0,3.0,20.5,1.0,0.0,0.0,0,12,14,6,0
4,3,2.0,0.0,2.1,0.7,43.0,4,14.0,5.0,16.4,1.0,0.0,0.0,1,17,12,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,24,1.0,0.0,0.5,0.9,50.0,12,7.0,3.0,14.9,1.0,0.0,0.0,0,8,20,4,19
1802,28,2.0,1.0,1.1,0.3,61.0,12,16.0,6.0,20.5,1.0,0.0,0.0,1,4,12,6,19
1803,29,0.0,2.0,0.9,3.2,38.0,12,15.0,3.0,17.6,0.0,0.0,0.0,0,17,14,6,19
1804,30,0.0,1.0,0.3,0.7,59.0,3,9.0,3.0,21.5,0.0,0.0,0.0,0,7,18,2,19


In [8]:
test_data = dataTest.iloc[:, :-3]
test_labels = dataTest.iloc[:, -3:]

In [9]:
test_data

,round,gf,ga,xg,xga,poss,formation,sh,sot,dist,fk,pk,pkatt,venue_code,opp_code,hour,day_code,home_code
0,0,2.0,0.0,1.0,1.2,44.0,15,10.0,2.0,14.6,1.0,0.0,0.0,0,6,20,4,0
1,11,4.0,2.0,2.7,0.5,50.0,15,19.0,7.0,13.0,0.0,0.0,0.0,1,10,15,5,0
2,22,3.0,0.0,1.3,0.3,57.0,15,14.0,6.0,14.8,0.0,0.0,0.0,0,2,17,5,0
3,32,2.0,1.0,2.6,0.8,71.0,15,22.0,8.0,15.5,1.0,0.0,0.0,1,8,17,5,0
4,33,2.0,1.0,2.4,0.4,59.0,15,22.0,8.0,16.3,1.0,0.0,0.0,1,1,19,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,19,1.0,2.0,1.1,1.8,57.0,12,7.0,4.0,19.5,1.0,0.0,0.0,0,14,16,6,19
605,20,2.0,4.0,2.7,1.6,65.0,15,23.0,8.0,16.1,1.0,0.0,0.0,1,9,15,5,19
606,21,1.0,1.0,0.5,1.3,71.0,12,8.0,1.0,17.2,0.0,0.0,0.0,0,15,15,5,19
607,23,1.0,0.0,0.6,1.1,38.0,17,9.0,4.0,24.2,0.0,0.0,0.0,1,5,15,5,19


In [10]:
train_data_reshape = train_data.values.reshape(-1, 1, 18)
train_data_reshape = np.reshape(train_data_reshape, (train_data_reshape.shape[0], -1, 1))
train_data_reshape

array([[[33.],
        [ 0.],
        [ 0.],
        ...,
        [13.],
        [ 6.],
        [ 0.]],

       [[35.],
        [ 2.],
        [ 0.],
        ...,
        [12.],
        [ 6.],
        [ 0.]],

       [[37.],
        [ 5.],
        [ 2.],
        ...,
        [13.],
        [ 6.],
        [ 0.]],

       ...,

       [[29.],
        [ 0.],
        [ 2.],
        ...,
        [14.],
        [ 6.],
        [19.]],

       [[30.],
        [ 0.],
        [ 1.],
        ...,
        [18.],
        [ 2.],
        [19.]],

       [[31.],
        [ 1.],
        [ 2.],
        ...,
        [16.],
        [ 6.],
        [19.]]])

In [11]:
test_data_reshape = test_data.values.reshape(-1, 1, 18)
test_data_reshape = np.reshape(test_data_reshape, (test_data_reshape.shape[0], -1, 1))
test_data_reshape

array([[[ 0.],
        [ 2.],
        [ 0.],
        ...,
        [20.],
        [ 4.],
        [ 0.]],

       [[11.],
        [ 4.],
        [ 2.],
        ...,
        [15.],
        [ 5.],
        [ 0.]],

       [[22.],
        [ 3.],
        [ 0.],
        ...,
        [17.],
        [ 5.],
        [ 0.]],

       ...,

       [[21.],
        [ 1.],
        [ 1.],
        ...,
        [15.],
        [ 5.],
        [19.]],

       [[23.],
        [ 1.],
        [ 0.],
        ...,
        [15.],
        [ 5.],
        [19.]],

       [[24.],
        [ 2.],
        [ 0.],
        ...,
        [15.],
        [ 5.],
        [19.]]])

In [12]:
test_labels_new = test_labels.values.reshape(-1, 3)
test_labels_new

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]], dtype=int64)

In [13]:
train_labels_new = train_labels.values.reshape(-1, 3)
train_labels_new

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=int64)

In [14]:
model = Sequential()
model.add(LSTM(units=64, input_shape=(18, 1)))
model.add(Dense(units=3, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [15]:
model.fit(train_data_reshape, train_labels_new, epochs=10, batch_size=18)

Epoch 1/10
101/101 [==============================] - 9s 82ms/step - loss: 0.6303 - accuracy: 0.3981
Epoch 2/10
101/101 [==============================] - 8s 81ms/step - loss: 0.6241 - accuracy: 0.4142
Epoch 3/10
101/101 [==============================] - 8s 82ms/step - loss: 0.6217 - accuracy: 0.4120
Epoch 4/10
101/101 [==============================] - 8s 82ms/step - loss: 0.6115 - accuracy: 0.4590
Epoch 5/10
101/101 [==============================] - 8s 82ms/step - loss: 0.6017 - accuracy: 0.4806
Epoch 6/10
101/101 [==============================] - 9s 85ms/step - loss: 0.5841 - accuracy: 0.5150
Epoch 7/10
101/101 [==============================] - 9s 88ms/step - loss: 0.5784 - accuracy: 0.5321
Epoch 8/10
101/101 [==============================] - 9s 87ms/step - loss: 0.5738 - accuracy: 0.5310
Epoch 9/10
101/101 [==============================] - 9s 86ms/step - loss: 0.5145 - accuracy: 0.6130
Epoch 10/10
101/101 [==============================] - 9s 84ms/step - loss: 0.3449 - accura

In [16]:
loss, accuracy = model.evaluate(test_data_reshape, test_labels_new)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

20/20 [==============================] - 1s 54ms/step - loss: 0.3033 - accuracy: 0.7980
Loss: 0.3033
Accuracy: 0.7980


In [17]:
newdata = [33, 0.0, 0.0, 1.4, 0.8, 49.0, 4, 11.0, 2.0, 17.5, 0.0, 0.0, 0.0, 0, 5, 13, 6, 0]

newdata_reshape = np.array(newdata).reshape(-1, 1, 18)
newdata_reshape = np.reshape(newdata_reshape, (newdata_reshape.shape[0], -1, 1))
newdata_reshape

array([[[33. ],
        [ 0. ],
        [ 0. ],
        [ 1.4],
        [ 0.8],
        [49. ],
        [ 4. ],
        [11. ],
        [ 2. ],
        [17.5],
        [ 0. ],
        [ 0. ],
        [ 0. ],
        [ 0. ],
        [ 5. ],
        [13. ],
        [ 6. ],
        [ 0. ]]])

In [18]:
predictNextNumber = model.predict(newdata_reshape)
print(predictNextNumber)

1/1 [==============================] - 0s 74ms/step
[[0.00344362 0.97783214 0.01872423]]


In [19]:
newdata2 = [35, 2.0, 0.0, 2.4, 0.4, 64.0, 4, 25.0, 8.0, 18.4, 2.0, 0.0, 0.0, 1, 4, 12, 6, 0]

newdata_reshape2 = np.array(newdata2).reshape(-1, 1, 18)
newdata_reshape2 = np.reshape(newdata_reshape2, (newdata_reshape2.shape[0], -1, 1))
newdata_reshape2

array([[[35. ],
        [ 2. ],
        [ 0. ],
        [ 2.4],
        [ 0.4],
        [64. ],
        [ 4. ],
        [25. ],
        [ 8. ],
        [18.4],
        [ 2. ],
        [ 0. ],
        [ 0. ],
        [ 1. ],
        [ 4. ],
        [12. ],
        [ 6. ],
        [ 0. ]]])

In [20]:
predictNextNumber2 = model.predict(newdata_reshape2)
print(predictNextNumber2)

1/1 [==============================] - 0s 65ms/step
[[0.98727906 0.00234263 0.01037827]]


In [21]:
# import pickle
# pickle.dump(model, open("../model/modelsoftmax.pkl", "wb"))